In [1]:
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import os
import pandas as pd
import glob
import math
import time
import sys
from scipy.spatial import cKDTree


In [2]:
def Make_dire(file_path):
    if not os.path.isdir(file_path):
        os.makedirs(file_path)

Dans NN, var en x pour simul y<br>
x : T, S, x, y, z<br>
y : MeltRate<br>
1ere Étape : Preparer les données

In [3]:
def get_depth_temp_sali(df, Data):
    ys, IceDraft = df.ny, df.iceDraft
    zs = np.array(Data.nz)
    N_T, N_S = [], []
    ylast = None
    dataset = Data[['temperatureYZ', 'salinityYZ']].to_dataframe().dropna()
    Index_Y = dataset.index.get_level_values('ny')
    for ind, y in enumerate(ys):
        indz = np.abs(np.array(zs + IceDraft[ind])).argmin()
        if y != ylast:
            Slice = dataset.iloc[Index_Y == y].droplevel('ny')
        Z_N = abs(np.array(Slice.index - indz)).argmin() + Slice.index[0]
        New_T = dataset.temperatureYZ.loc[Z_N, y]
        New_S = dataset.salinityYZ.loc[Z_N, y]
        N_T.append(New_T)
        N_S.append(New_S)
    return np.array(N_T), np.array(N_S)

In [4]:
def New_get_depth_temp_sal(df, Data): ## Method 3
    ys, Icedraft = np.array(df.ny), np.array(df.iceDraft)
    Var = ['temperatureYZ', 'salinityYZ']
    y = xr.DataArray(ys)
    iceZ = xr.DataArray(Icedraft)
    Data = Data.assign_coords({'nz':  - Data.nz})
    Filled = Data[Var].interpolate_na(dim="nz", use_coordinate = True, method = 'linear', fill_value="extrapolate")
    Cur = Filled[Var].interp(ny = y, nz = iceZ, method = 'linear')
    T, S = np.array(Cur.to_array())
    return T, S


In [5]:
def Method1_get_T_S(df, Data):
    ys = np.array(df.ny)
    y = xr.DataArray(ys)
    T = np.array(Data.temperatureYZ.sel(ny = y).mean(dim = 'nz', skipna = True))
    S = np.array(Data.salinityYZ.sel(ny = y).mean(dim = 'nz', skipna = True))
    return T, S

In [15]:
def Method2_get_T_S(df, Data):
    """
    Method returns for each point a list of size Z of values averaged on the Y dimensions
    """
    ys = np.array(df.ny)
    y = xr.DataArray(ys)
    zs = np.array(df.iceDraft)
    z = xr.DataArray(zs)
    
    Data = Data.assign_coords({'nz':  - Data.nz})
    T = Data.temperatureYZ.mean(dim = 'ny', skipna= True).interpolate_na(dim = 'nz', 
                                        method = 'nearest', fill_value = 'extrapolate').to_numpy()
    S = Data.salinityYZ.mean(dim = 'ny', skipna= True).interpolate_na(dim = 'nz', 
                                        method = 'nearest', fill_value = 'extrapolate').to_numpy()
    Names_T = [f"T_{i}" for i in range(Max)]
    Names_S = [f"S_{i}" for i in range(Max)]
    size = len(df)
    T = pd.DataFrame(np.tile(T, (size, 1)), columns = Names_T)
    S = pd.DataFrame(np.tile(S, (size, 1)), columns = Names_S)
    return pd.concat([T, S], axis=1)

In [21]:
def Method2p5_get_T_S(df, Data): ## Method 2.5
    
    ys = np.array(df.ny)
    y = xr.DataArray(ys)
    zs = np.array(df.iceDraft)
    z = xr.DataArray(zs)
    
    Data = Data.assign_coords({'nz':  - Data.nz})
    Var = ['temperatureYZ', 'salinityYZ']
    Cur = Data[Var].mean(dim = 'ny', skipna= True).interpolate_na(dim = 'nz', 
                                        method = 'nearest', fill_value = 'extrapolate')
    #T_s = Data.temperatureYZ.mean(dim = 'ny', skipna= True).interpolate_na(dim = 'nz', 
    #                                    method = 'nearest', fill_value = 'extrapolate')
    #S_s = Data.salinityYZ.mean(dim = 'ny', skipna= True).interpolate_na(dim = 'nz', 
    #                                    method = 'nearest', fill_value = 'extrapolate')
    
    Cur = Cur[Var].interp(nz = zs, method = 'linear')
    T, S = np.array(Cur.to_array())
    return T, S

In [8]:
def Method4_get_T_S(df, Data, auto_lite, extent = 40):
    """
    Method returns for each point a list of size Z of values at Y
    """
    ys = np.array(df.ny)
    y = xr.DataArray(ys)
    Data = Data.assign_coords({'nz':  - Data.nz})
    T = np.array(Data.temperatureYZ.sel(ny = y).interpolate_na(dim="nz", method = 'nearest', fill_value="extrapolate"))
    S = np.array(Data.salinityYZ.sel(ny = y).interpolate_na(dim="nz", method = 'nearest', fill_value="extrapolate"))
    
    Max = np.shape(T)[0]
    Names_T = [f"T_{i}" for i in range(Max)]
    Names_S = [f"S_{i}" for i in range(Max)]
    
    New_ind = np.linspace(start=0, stop=144, num = extent, endpoint=False).astype(int)
    NEW_T = [f"T_{i}" for i in New_ind]
    NEW_S = [f"S_{i}" for i in New_ind]
    
    DF_T = pd.DataFrame(T, index=Names_T).T
    DF_S = pd.DataFrame(S, index=Names_S).T
    if auto_lite:
        return pd.concat([DF_T[NEW_T], DF_S[NEW_S]], axis=1)
    else:
        return pd.concat([DF_T, DF_S], axis=1)

In [9]:
def Get_Ground_line(df, Arr):
    Arr = Arr.fillna(-9999)
    a = Arr.stack(grid=['ny', 'nx'])
    Grounds = a[a == -9999]
    Oc = a[a != -9999]
    inds_grounds = Grounds.indexes['grid'].to_frame().values.astype(float)
    inds_oc = df[['ny','nx']]
    Spat = cKDTree(inds_grounds)
    Dists, _ = Spat.query(inds_oc)
    return Dists

In [10]:
def Get_Front_line(df, Arr):
    a = Arr.stack(grid=['ny', 'nx'])
    Oc = a[a == 0]
    ice = a[a != -0]
    inds_oc = Oc.indexes['grid'].to_frame().values.astype(float)
    inds_ice = df[['ny','nx']]
    Spat = cKDTree(inds_oc)
    Dists, _ = Spat.query(inds_ice)
    return Dists

In [11]:
def Compute_slope(Data, df):
    ARR = Data
    ARR = xr.where(ARR < 100, 100, ARR).dropna('nx', 'all').interpolate_na(dim = "nx", use_coordinate = True, method = 'nearest', fill_value='extrapolate')
    x0, x1, y0, y1 = (ARR.shift(nx = -1, fill_value = np.nan).interpolate_na(dim = "nx", use_coordinate = True, method = 'nearest', fill_value='extrapolate'), 
                      ARR.shift(nx = 1, fill_value = np.nan).interpolate_na(dim = "nx", use_coordinate = True, method = 'nearest', fill_value='extrapolate'), 
                      ARR.shift(ny = -1, fill_value = np.nan).interpolate_na(dim = "ny", use_coordinate = True, method = 'nearest', fill_value='extrapolate'), 
                      ARR.shift(ny = 1, fill_value = np.nan).interpolate_na(dim = "ny", use_coordinate = True, method = 'nearest', fill_value='extrapolate'))
    dx = (x1 - x0)
    dy = (y1 - y0)
    d = (dx**2 + dy**2)**.5
    
    ys, xs = np.array(df.ny), np.array(df.nx)
    y, x = xr.DataArray(ys), xr.DataArray(xs)
    if len(d.coords) == 3 :
        d = d.sel(nx = x, ny = y).to_dataframe().drop(['nx', 'ny', 'nTime'], axis = 1)
        DX = dx.sel(nx = x, ny = y).to_dataframe().drop(['nx', 'ny', 'nTime'], axis = 1)
        DY = dy.sel(nx = x, ny = y).to_dataframe().drop(['nx', 'ny', 'nTime'], axis = 1)
    else : 
        d = d.sel(nx = x, ny = y).to_dataframe().drop(['nx', 'ny'], axis = 1)
        DX = dx.sel(nx = x, ny = y).to_dataframe().drop(['nx', 'ny'], axis = 1)
        DY = dy.sel(nx = x, ny = y).to_dataframe().drop(['nx', 'ny'], axis = 1)
    return d, DX, DY

In [19]:
File_path = '/bettik/bouissob/ISOMIP+/'
Bet_path = '/bettik/bouissob/'
Data_type = 'COM_NEMO-CNRS'
paths = glob.glob(File_path + f'*{Data_type}.nc')
paths

['/bettik/bouissob/ISOMIP+/Ocean1_COM_NEMO-CNRS.nc',
 '/bettik/bouissob/ISOMIP+/Ocean4_COM_NEMO-CNRS.nc',
 '/bettik/bouissob/ISOMIP+/Ocean3_COM_NEMO-CNRS.nc',
 '/bettik/bouissob/ISOMIP+/Ocean2_COM_NEMO-CNRS.nc']

In [13]:
File_path = '/bettik/bouissob/FAVIER_2019/'
Bet_path = '/bettik/bouissob/'
Data_type = 'COM_NEMO-CNRS'
paths = glob.glob(File_path + '*.nc')
paths

['/bettik/bouissob/FAVIER_2019/CPL_EXP13_rst_COM_NEMO-CNRS.nc',
 '/bettik/bouissob/FAVIER_2019/CPL_EXP23_rst_COM_NEMO-CNRS.nc',
 '/bettik/bouissob/FAVIER_2019/CPL_EXP11_rst_COM_NEMO-CNRS.nc',
 '/bettik/bouissob/FAVIER_2019/CPL_EXP21_rst_COM_NEMO-CNRS.nc',
 '/bettik/bouissob/FAVIER_2019/CPL_EXP10_rst_COM_NEMO-CNRS.nc',
 '/bettik/bouissob/FAVIER_2019/CPL_EXP20_rst_COM_NEMO-CNRS.nc',
 '/bettik/bouissob/FAVIER_2019/CPL_EXP22_rst_COM_NEMO-CNRS.nc',
 '/bettik/bouissob/FAVIER_2019/CPL_EXP12_rst_COM_NEMO-CNRS.nc']

In [23]:
Method = 2.5
for path in paths:
    if path.split('/')[-2] == 'FAVIER_2019':
        Name = path.split('/')[-1].replace(f'_{Data_type}.nc', '')
        auto_lite = False
        fav = True
    else:
        auto_lite = False
        Name = path.split('/')[-1].split('_')[0]
        fav = False
    dfT = pd.DataFrame()
    
    print(path)
    Data = xr.open_dataset(path)
    Data = Data.assign_coords({'nx': Data.x,
                           'ny': Data.y,
                           'nz': Data.z,
                           'nTime': Data.time})
    T = np.array(Data.nTime.as_numpy())
    Horiz_resol = 2 #km/index
    Time = len(T)
    Start = time.perf_counter()
    T = []
    df = []
    for t in range(Time):
        if (t+1)%int(Time/100) == 0:
            print('Starting {} / {}                       Elapsed time = {} s              ETA : {} s                         '.format(t+1, Time, 
                                                    int(time.perf_counter() - Start), int((Time-t+1) * (time.perf_counter() - Start)/(t+1))), end = '\r')
        df = pd.DataFrame()
        df['meltRate'] = (Data['meltRate'].isel(nTime = t).to_dataframe())['meltRate']
        df['thermalDriving'] = Data['thermalDriving'].isel(nTime = t).to_dataframe()['thermalDriving']
        df['halineDriving'] = Data['halineDriving'].isel(nTime = t).to_dataframe()['halineDriving']
        if Data['iceDraft'].ndim == 2:
            df['iceDraft'] = Data['iceDraft'].to_dataframe()['iceDraft']
            df['bathymetry'] = Data['bathymetry'].to_dataframe()['bathymetry']
#            df['bathymetry'] = Data['bathymetry'].to_dataframe()['bathymetry']
        else:
            df['iceDraft'] = Data['iceDraft'].isel(nTime = t).to_dataframe()['iceDraft']
            df['bathymetry'] = Data['bathymetry'].isel(nTime = t).to_dataframe()['bathymetry']
        df['date'] = t
        df.dropna(inplace = True)
        df.reset_index(inplace=True)

        if Data['iceDraft'].ndim == 2:
            if t == 0 :
                Dist_gl = Get_Ground_line(df, Data.iceDraft)
                Dist_fl = Get_Front_line(df, Data.iceDraft)
                Slope_ice, Slope_iceDraft_x, Slope_iceDraft_y = Compute_slope(Data.iceDraft, df)
                Slope_bathy, Slope_bathymetry_x, Slope_bathymetry_y = Compute_slope(Data.bathymetry, df)
            df['Distances_ground_line'] = Dist_gl
            df['Distances'] = Dist_gl
            df['Distances_front_line'] = Dist_fl
            df['Slope_iceDraft'] = Slope_ice
            df['Slope_iceDraft_x'] = Slope_iceDraft_x
            df['Slope_iceDraft_y'] = Slope_iceDraft_y
            
            df['Slope_bathymetry'] = Slope_bathy
            df['Slope_bathymetry_x'] = Slope_bathymetry_x
            df['Slope_bathymetry_y'] = Slope_bathymetry_y
        else :
            Dist_gl = Get_Ground_line(df, Data.iceDraft.isel(nTime = t))
            df['Distances_ground_line'] = Dist_gl
            df['Distances'] = Dist_gl
            df['Distances_front_line'] = Get_Front_line(df, Data.iceDraft.isel(nTime = t))
            df['Slope_iceDraft'], df['Slope_iceDraft_x'], df['Slope_iceDraft_y'] = Compute_slope(Data.iceDraft.isel(nTime = t), df)
            df['Slope_bathymetry'], df['Slope_bathymetry_x'], df['Slope_bathymetry_y'] = Compute_slope(Data.bathymetry.isel(nTime = t), df)
        
        if Method == 2:
            df_new = Method2_get_T_S(df, Data.isel(nTime = t))
            df = pd.concat([df, df_new], axis = 1)
        if Method == 3 or Method == None:
            T, S = New_get_depth_temp_sal(df, Data.isel(nTime = t))
            df['temperatureYZ'] = T
            df['salinityYZ'] = S
        if Method == 4:
            df_new = Method4_get_T_S(df, Data.isel(nTime = t), auto_lite)
            df = pd.concat([df, df_new], axis = 1)
            df.dropna(inplace = True)
        if Method == 2.5:
            T, S = Method2p5_get_T_S(df, Data.isel(nTime = t))
            df['temperatureYZ'] = T
            df['salinityYZ'] = S
        dfT = pd.concat([dfT, df], ignore_index = True)

    print('Done in {} s'.format(time.perf_counter() - Start))#, end = '\r')
    #dfT.dropna(inplace = True)
    #dfT.reset_index(drop = True, inplace = True)
    dfT.rename(columns={'nx' :'x', 'ny': 'y'}, inplace=True)
    if Method != None:
        if fav:
            f_path = Bet_path + f'Method_Data/FAVIER_2019/Method_{Method}/'
        else:
            f_path = Bet_path + f'Method_Data/{Data_type}/Method_{Method}/'
        Make_dire(f_path)
        print(sys.getsizeof(dfT) / 10**6)
        if auto_lite:
            pd.DataFrame.to_csv(dfT, f_path + f'{Name}_lite.csv', index = False)
        else:
            pd.DataFrame.to_csv(dfT, f_path + f'{Name}.csv', index = False)
#    else:
#        pd.DataFrame.to_csv(dfT, Bet_path + 'Data/New_data_{}_{}.csv'.format(Name, Data_type), index = False)

/bettik/bouissob/ISOMIP+/Ocean1_COM_NEMO-CNRS.nc
Done in 42.51501189300325 s              Elapsed time = 42 s              ETA : 0 s                          
69.79216
/bettik/bouissob/ISOMIP+/Ocean4_COM_NEMO-CNRS.nc
Done in 605.0905644720187 s                Elapsed time = 604 s              ETA : 1 s                          
398.57836
/bettik/bouissob/ISOMIP+/Ocean3_COM_NEMO-CNRS.nc
Done in 605.5849429899827 s                Elapsed time = 604 s              ETA : 1 s                          
384.14956
/bettik/bouissob/ISOMIP+/Ocean2_COM_NEMO-CNRS.nc
Done in 34.99909653200302 s              Elapsed time = 34 s              ETA : 0 s                          
82.87216


In [17]:
Name

'CPL_EXP23_rst'

In [ ]:
cpls = ['CPL_EXP10_rst', 'CPL_EXP12_rst', 'CPL_EXP20_rst', 'CPL_EXP22_rst',
'CPL_EXP11_rst', 'CPL_EXP13_rst', 'CPL_EXP21_rst', 'CPL_EXP23_rst'
]

In [ ]:
File_path = '/bettik/bouissob/ISOMIP+/'
Bet_path = '/bettik/bouissob/'
Data_type = 'COM_NEMO-CNRS'
Data_type = 'FAVIER_2019'
def Convert_to_lite(Bet_path, Data_type, Method, Desired_len = 40, Oc = 'Ocean1'):
    f_path = Bet_path + f'Method_Data/{Data_type}/Method_{Method}'
    path = f"{f_path}/{Oc}.csv"
    path = f"{f_path}/{Oc}_lite.csv"
    extent = 144
    
    df = pd.read_csv(path, nrows = 1)
    COL = df.columns
    New_ind = np.linspace(start=0, stop=extent, num = Desired_len, endpoint=False).astype(int)
    ALL_ind = [i for i in range(extent)]
    Dropping_ind = [i for i in ALL_ind if i not in New_ind]
    Drop_T = [f"T_{i}" for i in Dropping_ind]
    Drop_S = [f"S_{i}" for i in Dropping_ind]
    
    COL = [i for i in COL if ((i not in Drop_S) and (i not in Drop_T))]
    df = pd.read_csv(path, usecols = COL)
    T = [f"T_{i}" for i in New_ind]
    S = [f"S_{i}" for i in New_ind]
    df.rename(dict(zip(T, [ f"T_{i}" for i in np.arange(0, Desired_len).astype(str) ])), axis = 1, inplace = True)
    df.rename(dict(zip(S, [ f"S_{i}" for i in np.arange(0, Desired_len).astype(str) ])), axis = 1, inplace = True)
    pd.DataFrame.to_csv(df, f"{f_path}/{Oc}_lite.csv", index = False)

#for Oc in ['Ocean1', 'Ocean2', 'Ocean3', 'Ocean4']:
for Oc in cpls:
    print(f"Starting {Oc}")
    Start = time.perf_counter()
    A = Convert_to_lite(Bet_path, Data_type, Method = 4, Oc = Oc)
    print(f"{time.perf_counter() - Start} s elapsed")

In [4]:
File_path = '/bettik/bouissob/ISOMIP+/'
Bet_path = '/bettik/bouissob/'
Data_type = 'COM_NEMO-CNRS'
Method = 4
f_path = Bet_path + f'Method_Data/{Data_type}/Method_{Method}'
files = glob.glob(f"{f_path}/Ocean1_lite.csv")[0]